In [4]:
import torch
model_name = 'Llama-7B'
train_data = torch.load('/home/xujiaming/xujiaming/research/ASPLOS-24/results/'+model_name+'_feature.pt')
train_label = torch.load('/home/xujiaming/xujiaming/research/ASPLOS-24/results/'+model_name+'_label.pt')

In [4]:
import torch
import torch.nn as nn
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out
    


In [5]:
model = torch.load('/home/xujiaming/xujiaming/research/ASPLOS-24/results/model1.pth')

In [28]:
model.fc1.weight.dtype

torch.float16

In [21]:
x = torch.rand(1,4096+8).to(model.fc1.weight.device)
x

tensor([[0.3903, 0.7349, 0.9442,  ..., 0.2022, 0.5751, 0.8548]],
       device='cuda:0')

In [22]:
model(x)

tensor([[0.0065]], device='cuda:0', grad_fn=<SigmoidBackward0>)

In [18]:
train_label = torch.load('/home/xujiaming/xujiaming/research/ASPLOS-24/results/train_label.pt')

In [13]:
print(train_label)

tensor([0, 0, 0,  ..., 1, 1, 1], device='cuda:0', dtype=torch.int32)


In [8]:
from eagle.model.ea_model import EaModel
from fastchat.model import get_conversation_template
import torch
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from typing import Optional
import json
import time 

model = EaModel.from_pretrained(
    base_model_path='/share/datasets/public_models/Llama-2-7b-chat-hf',
    ea_model_path="/home/xujiaming/xujiaming/datasets/LLM_models/EAGLE-llama2-chat-7B",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    device_map="auto",
    # is_offload = False,
    # skip_model = "/home/xujiaming/xujiaming/research/ASPLOS-24/skip_layer/model.txt",
)

/share/xujiaming/anaconda3/envs/spec_de/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-06-26 22:28:37,906] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Loading checkpoint shards: 100%|██████████| 2/2 [00:31<00:00, 15.74s/it]


In [13]:
model.base_model.lm_head.weight[[[[1,23,3222,3432]]]].shape

torch.Size([1, 4, 4096])

In [14]:
a = torch.arange(12)

In [15]:
a

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [16]:
top=torch.topk(a, 3, dim=-1)

In [17]:
top

torch.return_types.topk(
values=tensor([11, 10,  9]),
indices=tensor([11, 10,  9]))

In [ ]:


for x in range(18,32):
    max_acc = 0
    min_err = 100000
    threshold = 19.75
    while threshold < 30:
        acc = 0
        err = 0
        for i in range(x,train_data.shape[0],32):
            max_prob = torch.max(train_data[i],dim=-1).values
            if(max_prob > threshold and train_label[i] == 1):
                acc += 1
            if max_prob > threshold and train_label[i] == 0:
                err += 1
        tot = torch.sum(train_label[x:-1:32])
        acc_ratio= acc / tot
        # if acc > max_acc:
        #     max_acc = acc
        #     threshold_set_err[x] = threshold
        if err < min_err:
            min_err = err
            threshold_set_err[x] = threshold
        threshold += 0.25
        
    print(x,threshold_set_err[x],acc_ratio,int(acc),'/',int(tot),int(err),'/',int(tot))
        


In [ ]:
threshold_ratio = 2
threshold = 6.5
# for x in range(32):
x = 0
acc = 0
err = 0
for i in range(x,train_data.shape[0],32):
    result_descend = train_data[i].sort(descending=True).values
    result_descend_soft = torch.nn.functional.softmax(result_descend,dim=-1)
    top_logits = result_descend[0]
    second_logits = result_descend[1]
    top_prob = result_descend_soft[0]
    second_prob = result_descend_soft[1]
    if(top_prob / second_prob > threshold_ratio and top_logits / second_logits > threshold_ratio and top_logits > threshold):
        if train_label[i] == 1:
            acc += 1
        else:
            print(result_descend,result_descend_soft,train_label[i])
            err += 1
tot = torch.sum(train_label[x:-1:32])
acc_ratio = acc / tot
print(x,acc_ratio,int(acc),'/',int(tot),int(err),'/',int(tot),)

In [16]:
a = torch.tensor([1,2,3,4],dtype=torch.float32)
a = a.sort(descending=True).values

In [17]:
torch.nn.functional.softmax(a)

/tmp/ipykernel_2826536/2430103232.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  torch.nn.functional.softmax(a)


tensor([0.6439, 0.2369, 0.0871, 0.0321])

In [3]:
import torch.nn.functional as F
import torch

In [4]:
a = torch.ones((1,100))
b = torch.ones((4,100))

In [6]:
F.linear(a,b).shape

torch.Size([1, 4])

In [1]:
sum([1,2,3,3])

9